## Ввод датасета/нашей документации

In [1]:
from striprtf.striprtf import rtf_to_text
import os
from nltk.tokenize import LineTokenizer

In [2]:
ltokenizer=LineTokenizer()

In [3]:
def get_rtf_names():
    rtf_names=[]
    names=os.listdir()
    for name in names:
        if ".rtf" in name:
            rtf_names.append(name)
    return rtf_names

In [4]:
def get_text_doc(fname):
    file=open(fname, 'r')
    rtfdoc=file.read()
    document=rtf_to_text(rtfdoc)
    return document

In [5]:
def parse_instruction(document, tokenizer=ltokenizer):
    lines=tokenizer.tokenize(document)
    tok_begin="Должностные обязанности"
    tok_end="Права"
    i_begin=0
    i_end=0
    cnt=0
    for l in lines:
        if tok_begin in l:
            i_begin=cnt+1
        if tok_end in l:
            i_end=cnt
        cnt+=1
    hline=lines[i_begin]
    main_cont=lines[i_begin+1:i_end]
    the_rest=lines[0:i_begin]+lines[i_end+1:-1]
    return hline, main_cont, the_rest

In [6]:
def get_data_text(paragraph_breaker=ltokenizer):
    headers=[]
    data_text=[]
    secondary_text=[]
    rtf_names=get_rtf_names()
    for name in rtf_names:
        document=get_text_doc(name)
        hline, main_cont, the_rest=parse_instruction(document,tokenizer=paragraph_breaker)
        for k in range(0,len(main_cont)):
            headers.append(hline)
        data_text+=main_cont
        secondary_text+=the_rest
    return headers, data_text, secondary_text

In [7]:
def prepare_text(init_text, tokenizer, stop_words, morpher):
    init_text=init_text.lower()
    tokens=tokenizer.tokenize(init_text)
    tokens_clear=[]
    for t in tokens:
        if t in stop_words:
            continue
        tokens_clear.append(t)
    tokens_lemmatized=[]
    for t in tokens_clear:
        p = morph.parse(t)[0]
        p=p.normal_form
        tokens_lemmatized.append(p)
    return tokens_lemmatized

In [30]:
def get_top_docs(init_text, headers, data_text, REtokenizer, stop_words, morpher, model, n=5):
    tokens_lemmatized=prepare_text(init_text, REtokenizer, stop_words, morpher)
    sim_ind=[]
    cnt=0
    for text in data_text:
        doc=prepare_text(text,REtokenizer, stop_words, morpher)
        if (len(doc)==0 or len(tokens_lemmatized)==0):
            continue
        sim=model.n_similarity(tokens_lemmatized, doc)
        sim_ind.append((sim,cnt))
        cnt+=1
    sim_ind.sort(reverse=True)
    top_docs=[]
    top_headers=[]
    if n>len(sim_ind):
        return
    for i in range(0,n):
        j=sim_ind[i][1]
        top_docs.append(data_text[j])
        if headers[j] not in top_headers:
            top_headers.append(headers[j])
    return top_docs, headers

## Готовим инструменты

In [9]:
import re
import logging

import gensim.downloader as api
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer

import pymorphy2

#from nltk import pos_tag_sents
#from nltk.tokenize import word_tokenize
#nltk.download('punkt')
#nltk_download('averaged_perceptron_tagger_ru')
#nltk.download('universal_tagset')
#pos_tag_sents(word_tokenize("настоящей должностной инструкцией"), tagset='universal',lang='rus')

## Подключаем gensim и загружаем векторную модель слов (уже готовую, model.model)

In [10]:
import gensim

In [11]:
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

In [12]:
api.BASE_DIR='C:\\Users\\Home\\gensim-data\\'

In [13]:
api.BASE_DIR

'C:\\Users\\Home\\gensim-data\\'

In [14]:
#model=api.load("word2vec-ruscorpora-300")

In [15]:
md=gensim.models.KeyedVectors.load("C:\\Users\\Home\\gensim-data\\model.model") #C:\\Users\\Home\\gensim-data\\model.model

INFO - 20:25:55: loading Word2VecKeyedVectors object from C:\Users\Home\gensim-data\model.model
INFO - 20:25:56: loading vectors from C:\Users\Home\gensim-data\model.model.vectors.npy with mmap=None
INFO - 20:25:56: loading vectors_ngrams from C:\Users\Home\gensim-data\model.model.vectors_ngrams.npy with mmap=None
INFO - 20:25:57: loading vectors_vocab from C:\Users\Home\gensim-data\model.model.vectors_vocab.npy with mmap=None
INFO - 20:25:57: setting ignored attribute vectors_ngrams_norm to None
INFO - 20:25:57: setting ignored attribute vectors_norm to None
INFO - 20:25:57: setting ignored attribute buckets_word to None
INFO - 20:25:57: setting ignored attribute vectors_vocab_norm to None
INFO - 20:25:57: loaded C:\Users\Home\gensim-data\model.model


## Получаем входной текст и предобрабатываем его: леммтизация, токенизация, выкидывание стоп-слов, частых/редких слов

In [44]:
init_text="Генеральный директор, руководство персоналом, работа с документами"

In [17]:
stop_words = stopwords.words('russian')

### Разбиваем текст на отдельные токены (здесь, по сути, на слова)

In [18]:
tokenizer = RegexpTokenizer('\w+|[^\w\s]+\.') # с паттернами сложно, надо отрегулировать #'\w+|[^\w\s]+'

In [19]:
morph = pymorphy2.MorphAnalyzer()

INFO - 20:25:57: Loading dictionaries from C:\local\Anaconda3\lib\site-packages\pymorphy2_dicts\data
INFO - 20:25:57: format: 2.4, revision: 393442, updated: 2015-01-17T16:03:56.586168


In [45]:
tokens_lemmatized=prepare_text(init_text, tokenizer, stop_words, morph)

## Получаем входную базу текста

In [21]:
headers, data_text, the_rest=get_data_text()

## Получаем рекомендуемый текст и заголовки

In [46]:
top_docs, top_hdr=get_top_docs(init_text, headers, data_text, tokenizer, stop_words,morph, md)

In [47]:
top_docs

[' 1. Осуществляет руководство работой персонала машинописного бюро.',
 ' 1.2. Передача докладных записок после их регистрации заведующему канцелярией (начальнику отдела документационного обеспечения) для доклада руководству организации.',
 '2. Принимает поступающую на рассмотрение руководителя корреспонденцию, передает ее в соответствии с принятым решением в структурные подразделения или конкретным исполнителем для использования в процессе работы либо подготовки ответов.',
 '3. Руководит работой по подготовке необходимых отчетов и справок, связанных с деятельностью бюро и выдачей пропускных документов.',
 'Оказывать помощь коллегам по работе при решении задач их деятельности в случае, если помощь может привести к качественному улучшению результатов деятельности. ',
 'Участвовать в реализации мероприятий по формированию и транслировании корпоративной культуры и улучшению социально-психологического климата в коллективе. ',
 '4. Планирует внеклассную и внешкольную воспитательную работу, 

In [39]:
#top_hdr